<a href="https://colab.research.google.com/github/neslhan00/dsai301/blob/main/Wordle_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neslihan Gül - November 2024

Wordle Solver Project

My goal in this project was to use algorithmic techniques and data analysis to solve Wordle as efficiently as possible.
I took a methodical approach to this issue, first dissecting the structure of the game, then applying logical filters, and lastly setting the plan through iterative testing.

In [2]:
# I imported necessary libraries.
import requests
import random
import time

In [3]:
def download_file_from_drive(file_url, colab_path): #Downloads a file from Google Drive and saves it locally in Google Colab. Used to fetch the word list for the Wordle solver.
    if "drive.google.com" in file_url:
        file_id = file_url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        raise ValueError("The provided URL is not a valid Google Drive link.")

    response = requests.get(download_url, stream=True)

    if response.status_code == 200:
        with open(colab_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File successfully downloaded to {colab_path}")
    else:
        raise Exception("Failed to download the file. Check the URL or file permissions.")

colab_path = "/content/words.txt"
file_url = "https://drive.google.com/file/d/1cjCZ-5uAHVFLXLV6HuSM98_QblWaXrBZ/view?usp=share_link"
download_file_from_drive(file_url,colab_path)

WORDLIST_FILENAME = "words.txt"

def load_words():   #Reads words from a file and returns them as a list.

  inFile = open(WORDLIST_FILENAME, 'r')
  line = inFile.readline()
  wordlist = line.split()
  print("  ", len(wordlist), "words loaded.")
  return wordlist




def word_select(word_list):   #Randomly selects a word from the word list.
  return random.choice(word_list)

File successfully downloaded to /content/words.txt


In [4]:
# Helper functions

def updatelist1(list1,letter):    #Adds a letter to the list of incorrect guesses.
  if letter not in list1:  # Ensures the letter is not added multiple times
    list1.append(letter)
  return list1

def updatelist2(list2,update):   #Updates the list of correct letters in the correct position.
  for item in list2:
    if item[0]==update[0]:
      return list2
  list2.append(update)
  return list2

def updatelist3(list3,update):   #Updates the list of correct letters in the wrong position.
  for item in list3:
    if item[0]==update[0]:
      if update[1] not in item[1]:
        item[1].append(update[1][0])
        return list3
      return list3
  list3.append(update)
  return list3

def cleanwordlist(word_list,secretword):   #Filters words to match the length of the secret word.
  for word in word_list[:]:
    if len(word)!=len(secretword):
      word_list.remove(word)
  return word_list




  """
  Wordle function:
  Compares a guess to the secret word and categorizes letters into three lists:
  - list1: Incorrect letters
  - list2: Correct letters in the correct position
  - list3: Correct letters in the wrong position
  """
def wordle(guess, secretword):

  list1=[]
  list2=[]
  list3=[]
  iscorrect=False
  if guess==secretword:
    bildimi=True   # Checks if the guess is correct
    return list1,list2,list3,bildimi
  for index in range(len(guess)):
    if guess[index]==secretword[index]:
      list2 = updatelist2(list2,[guess[index],index])  # Correct letter in correct position
    elif guess[index] not in secretword:
      list1=updatelist1(list1,guess[index])  # Letter is not in the word
    elif guess[index] in secretword:
      list3=updatelist3(list3,[guess[index],[index]])  # Correct letter in wrong position
  return list1,list2,list3,iscorrect


def generate_guess(list1,list2,list3,wordlist):   # Generates the next AI guess by eliminating impossible words.
  for word in wordlist[:]:
    for letter in list1:
      if letter in word:
        wordlist.remove(word)
        break
    if word  not in wordlist:
      continue
    for item in list2:
      if word[item[1]]!=item[0]:
        wordlist.remove(word)
        break
    if word  not in wordlist:
      continue
    for item in list3:
      if item[0] not in word:
        wordlist.remove(word)
        break
    if word  not in wordlist:
      continue
    for item in list3:
      for index in item[1]:
        if word[index]==item[0]:
          wordlist.remove(word)
          break
      if word not in wordlist:
        break
  guess=random.choice(wordlist)
  return guess



def optimize_wordle():   # Runs the AI Wordle solver until the correct word is found.
  word_list=load_words()
  secretword=word_select(word_list)
  print(f"Secretword is :{secretword}\nComputer will try to find this word...")
  word_list=cleanwordlist(word_list,secretword)
  list1=[]
  list2=[]
  list3=[]
  guesses=[]
  while True:
    guess=generate_guess(list1,list2,list3,word_list)  # AI makes a guess
    guesses.append(guess)
    print(f"Computer's guess is: {guess}")
    list1,list2,list3,iscorrect=wordle(guess,secretword)
    time.sleep(2)  # Adds delay for readability
    if iscorrect:
      print(f"The computer guessed the word in {len(guesses)} attempts.\nList of the computer's guesses:{guesses}")
      break
    if secretword not in word_list:
      print(guesses)
      print("Word couldn't find.")
      break

In [5]:
optimize_wordle()

   55900 words loaded.
Secretword is :philosophy
Computer will try to find this word...
Computer's guess is: billboards
Computer's guess is: philosophy
The computer guessed the word in 2 attempts.
List of the computer's guesses:['billboards', 'philosophy']
